In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import io
import imageio
from IPython.display import Image, display
from ipywidgets import widgets, Layout, HBox

In [2]:
import os
import numpy as np
import natsort
from PIL import Image

file_path = './heatmap_case4_0.2'
file_list = os.listdir(file_path)
file_list = natsort.natsorted(file_list, key=None, reverse=False, alg=0)

num_of_files = len(file_list)

# Assuming all images have the same height and width
first_image = Image.open(os.path.join(file_path, file_list[0]))
height, width = np.array(first_image).shape

# Create an empty NumPy array to store the images
image_array = np.empty((num_of_files, height, width))

# Iterate through the file list and load images
for i, file_name in enumerate(file_list):
    image = Image.open(os.path.join(file_path, file_name))
    image_array[i] = np.array(image)

# image_array now contains the images as NumPy arrays
print(image_array.shape)  # (num_of_files, height, width)
print(file_list[:5])

(1300, 9, 64)
['train0.png', 'train1.png', 'train2.png', 'train3.png', 'train4.png']


In [3]:
import os
import numpy as np
from PIL import Image

window_size = int(input())
num_of_frames = window_size+1  # Specify the desired number of frames

# Assuming all images have the same height and width
first_image = Image.open(os.path.join(file_path, file_list[0]))
height, width = np.array(first_image).shape[:2]

# Create an empty NumPy array to store the images
dataset = np.empty((num_of_files-num_of_frames+1, num_of_frames, height, width))

# Iterate through the file list and load images
for i in range(num_of_files-num_of_frames+1):
    frame_images = file_list[i : i+num_of_frames]
    for j, file_name in enumerate(frame_images):
        if i < 2:
            print(file_name)
        image = Image.open(os.path.join(file_path, file_name))
        dataset[i, j] = np.array(image)
    if i < 2:
        print()

# dataset now contains the images bound into frames
print(dataset.shape)  # (num_of_files // num_of_frames, num_of_frames, height, width)

 12


train0.png
train1.png
train2.png
train3.png
train4.png
train5.png
train6.png
train7.png
train8.png
train9.png
train10.png
train11.png
train12.png

train1.png
train2.png
train3.png
train4.png
train5.png
train6.png
train7.png
train8.png
train9.png
train10.png
train11.png
train12.png
train13.png

(1288, 13, 9, 64)


In [4]:
# Add a channel dimension since the images are grayscale.
dataset = np.expand_dims(dataset, axis=-1)
dataset.shape #(num_of_samples, frames, height, width, channels)

(1288, 13, 9, 64, 1)

In [5]:
# Split into train and validation sets using indexing to optimize memory.
indexes = np.arange(dataset.shape[0])
np.random.shuffle(indexes)
train_index = indexes[: int(0.9 * dataset.shape[0])]
val_index = indexes[int(0.9 * dataset.shape[0]) :]
train_dataset = dataset[train_index]
val_dataset = dataset[val_index]

# Normalize the data to the 0-1 range.
train_dataset = train_dataset / 255
val_dataset = val_dataset / 255

# We'll define a helper function to shift the frames, where
# `x` is frames 0 to n - 1, and `y` is frames 1 to n.
def create_shifted_frames(data):
    x = data[:, 0 : data.shape[1] - 1, :, :]
    y = data[:, 1 : data.shape[1], :, :]
    return x, y


# Apply the processing function to the datasets.
x_train, y_train = create_shifted_frames(train_dataset)
x_val, y_val = create_shifted_frames(val_dataset)

# Inspect the dataset.
print("Training Dataset Shapes: " + str(x_train.shape) + ", " + str(y_train.shape))
print("Validation Dataset Shapes: " + str(x_val.shape) + ", " + str(y_val.shape))

Training Dataset Shapes: (1159, 12, 9, 64, 1), (1159, 12, 9, 64, 1)
Validation Dataset Shapes: (129, 12, 9, 64, 1), (129, 12, 9, 64, 1)


# Data Visualization

In [6]:
# # Construct a figure on which we will visualize the images.
# fig, axes = plt.subplots(2, 6, figsize=(10, 8))

# # Plot each of the sequential images for one random data example.
# np.random.seed(42)
# data_choice = np.random.choice(range(len(train_dataset)), size=1)[0]
# for idx, ax in enumerate(axes.flat):
#     ax.imshow(np.squeeze(train_dataset[data_choice][idx]), cmap="gray")
#     ax.set_title(f"Frame {idx + 1}")
#     ax.axis("off")

# # Print information and display the figure.
# print(f"Displaying frames for example {data_choice}.")
# plt.show()

# Model Construction

To build a Convolutional LSTM model, we will use the
`ConvLSTM2D` layer, which will accept inputs of shape
`(batch_size, num_frames, width, height, channels)`, and return
a prediction movie of the same shape.

In [7]:
# Construct the input layer with no definite frame size.
inp = layers.Input(shape=(None, *x_train.shape[2:]))

# We will construct 3 `ConvLSTM2D` layers with batch normalization,
# followed by a `Conv3D` layer for the spatiotemporal outputs.
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(5, 5),
    padding="same",
    return_sequences=True,
    activation="relu",
)(inp)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(3, 3),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(1, 1),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)
x = layers.Conv3D(
    filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
)(x)

# Next, we will build the complete model and compile it.
convlstm = keras.models.Model(inp, x)
convlstm.compile(
    loss=tf.keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(),
)

# Model Training

With our model and data constructed, we can now train the model.

In [8]:
# Define some callbacks to improve training.
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5)

# Define modifiable training hyperparameters.
epochs = 20
batch_size = 5

In [9]:
# Fit the model to the training data.

with tf.device("/gpu:3"):
    convlstm.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(x_val, y_val),
        callbacks=[early_stopping, reduce_lr],
    )

Epoch 1/20
232/232 [==============================] - 98s 390ms/step - loss: 0.0092 - val_loss: 0.0425
Epoch 2/20
232/232 [==============================] - 92s 397ms/step - loss: 0.0017 - val_loss: 0.0188
Epoch 3/20
232/232 [==============================] - 90s 390ms/step - loss: 0.0013 - val_loss: 0.0028
Epoch 4/20
232/232 [==============================] - 91s 392ms/step - loss: 0.0012 - val_loss: 0.0014
Epoch 5/20
232/232 [==============================] - 92s 397ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 6/20
232/232 [==============================] - 92s 395ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 7/20
232/232 [==============================] - 91s 392ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 8/20
232/232 [==============================] - 91s 391ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 9/20
232/232 [==============================] - 91s 394ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 10/20
232/232 [==============================] - 91s 393ms/step - l

In [10]:
convlstm.save('./convlstm_models/convlstm_windowsize'+str(window_size)+'_'+file_path[-9:])

INFO:tensorflow:Assets written to: ./convlstm_models/convlstm_windowsize12_case4_0.2\assets


# Frame Prediction Visualizations

With our model now constructed and trained, we can generate
some example frame predictions based on a new video.

We'll pick a random example from the validation set and
then choose the first ten frames from them. From there, we can
allow the model to predict 10 new frames, which we can compare
to the ground truth frame predictions.

- model predict 시 input shape : ()

In [ ]:
def predict_next_frames(model, example, num_of_frames):

    # Extract the model's prediction and post-process it.
    new_prediction = model.predict(np.expand_dims(example[:-1], axis=0))
    new_prediction = np.squeeze(new_prediction, axis=0)
    predicted_frame = np.expand_dims(new_prediction[-1, ...], axis=0)

    print('model input shape : ', np.expand_dims(example[:-1], axis=0).shape)
    print('model output shape : ', predicted_frame.shape)

    return predicted_frame

In [ ]:
K = 16
K_list = [8, 16, 32, 64]

def find_top_k_indices(matrix):
    result = []
    for row in matrix:
        mecs_top_k = []
        # (인덱스, 값) 쌍으로 이루어진 리스트를 생성합니다.
        indices_values = [(index, value) for index, value in enumerate(row)]
        #print(indices_values)
        # 값에 따라 정렬합니다. (큰 값이 먼저 오도록)
        sorted_indices_values = sorted(indices_values, key=lambda x: x[1], reverse=True)
        # 상위 32개의 인덱스를 결과에 추가합니다.
        top_indices = [index for index, _ in sorted_indices_values[:K]]
        mecs_top_k.extend(top_indices)
        result.append(mecs_top_k)
    return result


In [ ]:
np.random.seed(42)
total_hits_list = [[] for _ in range(x_train.shape[2])] # mecs수 만큼 빈 객체 생성

for val_index in range(val_dataset.shape[0]): # 모든 val_dataset에 대해 predict
    example = val_dataset[val_index]

    print('val_dataset shape : ', val_dataset.shape)
    print(val_index,'번째 example shape : ', example.shape)

    original_frame = example[-1,...]
    new_prediction = predict_next_frames(model, example, num_of_frames)

    original_frame = np.squeeze(original_frame)
    new_prediction = np.squeeze(new_prediction)

    top_k_truths = find_top_k_indices(original_frame)
    top_k_predicts = find_top_k_indices(new_prediction)

    total_hits = 0
    

    for idx, (truths_mecs, predict_mecs) in enumerate(zip(top_k_truths, top_k_predicts)):
        hits_per_mecs = sum([1 for i in truths_mecs if i in predict_mecs])
        total_hits_list[idx].append(hits_per_mecs)
        total_hits += hits_per_mecs

In [ ]:
val_dataset.shape

In [ ]:
print('hit rate log 개수 : ', len(total_hits_list))
print('hit rate log per mec : ', len(total_hits_list[0]))

In [ ]:
import numpy as np

total_hits_rate = 0
total_hits_per_mecs = []

for hits_per_mecs in total_hits_list:
    total_hits_per_mecs.append(sum(hits_per_mecs))

total_hits_rate_per_mecs = [total_hits_per_mec / val_dataset.shape[0] for total_hits_per_mec in total_hits_per_mecs]# mecs 수로 나누어줌
total_hits_rate = sum(total_hits_per_mecs) / (val_dataset.shape[0] * val_dataset.shape[2])

print('mecs 별 평균 : ', total_hits_rate_per_mecs)
print('전체 mecs 평균 : ', total_hits_rate)

In [ ]:
hits_rate_per_mecs = (np.array(total_hits_rate_per_mecs) / K) * 100

print('hit rate per mec : ', hits_rate_per_mecs)
print('hit rate total mec (mean/variance/std) : ', hits_rate_per_mecs.mean(), hits_rate_per_mecs.var(),hits_rate_per_mecs.std())

# Boxplot 시각화

In [ ]:
total_hits_rate_list = (np.array(total_hits_list) / K) * 100
total_hits_rate_list

In [ ]:
total_hits_rate_list = list(total_hits_rate_list)

In [ ]:
import matplotlib.pyplot as plt

# Assuming total_hits_list is a 2D list with 16 rows and 143 columns
# Each row represents the hit rate data for one MEC.

# Create a list of labels for the 16 MECs (e.g., MEC_1, MEC_2, ..., MEC_16)
mec_labels = [f"{i}" for i in range(image_array.shape[1])]

# Create a figure and axis object for the boxplot
fig, ax = plt.subplots()

# Create the boxplot using the boxplot function
ax.boxplot(total_hits_rate_list, labels=mec_labels)

# Set axis labels and title
ax.set_xlabel('MECs index')
ax.set_ylabel('Hit Rate(%)')
#ax.set_title('Hit Rate Distribution for '+str(val_dataset.shape[2])+ ' MECs')

# Save the plot.png
plt.savefig('./Figs./'+file_path[-9:]+'convlstm_K='+str(K)+'.png')

# Show the plot
plt.show()